In [2]:
import cassandra
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
import os
import pandas as pd
import fiddler as fdl

In [3]:
cassandra.__version__

'3.28.0'

In [4]:
# This secure connect bundle is autogenerated when you donwload your SCB, 
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'datastax_auth/secure-connect-fiddlerai.zip'
}

ASTRA_DB_APPLICATION_TOKEN = os.environ.get('ASTRA_DB_APPLICATION_TOKEN')
#print("TOKEN: " + ASTRA_DB_APPLICATION_TOKEN)

# This token json file is autogenerated when you donwload your token, 
# if yours is different update the file name below
#with open("datastax_auth/danny@fiddler.ai-token.json") as f:
#    secrets = json.load(f)

#CLIENT_ID = secrets["clientId"]
#CLIENT_SECRET = secrets["secret"]

auth_provider=PlainTextAuthProvider("token", ASTRA_DB_APPLICATION_TOKEN)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)

#auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
#cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

In [5]:
session.set_keyspace('fiddlerai')

In [6]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

In [7]:
rows = session.execute('SELECT * FROM fiddler_rfp_snippets_openai')
    
df_docs = rows._current_rows
df_docs

,row_id,attributes_blob,body_blob,metadata_s,vector
0,972c0474427c4237872c3bbbab1951e4,None,c) Will you notify us of any such requests and...,None,"[0.0036626053042709827, -0.02872580476105213, ..."
1,68945f36a3b1461697a373973b8a801b,None,Model Operations_002\nFiddler tracks model dri...,None,"[-0.040573589503765106, -0.002487645950168371,..."
2,3cf33350903f433f80e147a9a3539010,None,"If yes, please explain any methods used to phy...",None,"[0.008808407001197338, 0.012253711931407452, 0..."
3,9179821238164c7caf24df1a78509d48,None,Is there a clearly defined acceptable use poli...,None,"[0.010075459256768227, -0.016981706023216248, ..."
4,97f5fb53b11a444e8d87f95d3cd219b5,None,What are the technical requirements for client...,None,"[0.01567554660141468, -0.007403489667922258, 0..."
...,...,...,...,...,...
1122,9b2d1b9dbd544d74bf656bf22a985f4b,None,"Upon destruction/shredding of Unity material, ...",None,"[-0.0013801063178107142, -0.019234688952565193..."
1123,4a4d54d767a149b3a9b90e52ad25aee6,None,Architecture_017\nModel instrumentation events...,None,"[-0.020267708227038383, -0.003999302629381418,..."
1124,43c61d0601ff404ab4ef362313805b8f,None,Model Monitoring_018\nThe inference data can b...,None,"[-0.028017668053507805, -0.011126500554382801,..."
1125,5eba4dbfd4254faa843633d495c9b56a,None,How and when are files deleted? Please provide...,None,"[-0.01652630977332592, -0.003884867299348116, ..."


In [9]:
rows = session.execute('SELECT row_id, session_id, source_docs, response, question, comment, feedback, ts FROM fiddler_rfp_history')
    
df = rows._current_rows
df

,row_id,session_id,source_docs,response,question,comment,feedback,ts
0,cfd18c75-b400-40c3-92f1-308ba0e24c95,8141d863-cd10-43db-b68a-4f0ee8898ddd,"Document: If AWS or Azure, what cloud servi...","Yes, Fiddler is deployed into AWS. \n\nReferen...",Does Fiddler deploy into AWS?,None,None,2023-11-20 21:36:54.941
1,2e7fd0f8-02a4-464f-a949-29bf04fc8be7,8141d863-cd10-43db-b68a-4f0ee8898ddd,Document: How does Fiddler encrypt data at ...,"Yes, Fiddler encrypts data at rest using AWS A...",Does Fiddler encrypt data at rest?,None,None,2023-11-20 21:37:11.195
2,6c4b6b34-c333-4786-8bfc-28d1975f62f7,8141d863-cd10-43db-b68a-4f0ee8898ddd,Document: Do you provide IP allowlisting or...,"Yes, Fiddler can implement IP whitelisting at ...",Does Fiddler whitelist IPs?,None,None,2023-11-20 21:36:39.113


In [16]:
#session.execute("DELETE FROM fiddler_chatbot_conversation WHERE row_id='-1'")

In [35]:
#session.execute("DROP TABLE fiddler_chatbot_history")

In [10]:
session.execute("TRUNCATE TABLE fiddler_rfp_snippets_openai")

In [27]:
session.execute("TRUNCATE TABLE fiddler_rfp_history")